# 8-1. 들어가며
머신러닝 어플리케이션 중 가장 상업적인 성공을 거둔 것으로 평가받고 있는 추천시스템(Recommender System)에 대해  
이런 추천시스템의 원리를 한 문장으로 줄인다면 나와 비슷한 다른 사용자들이 좋아하는 것과 비슷한 것을 내게 추천해 준다 일 것입니다.   
하지만 어떤 사용자들이 나와 비슷한지를 어떻게 알 수 있을까요? 또 어떤 상품이나 정보가 서로 유사한지는 어떻게 알 수 있는 걸까요?   
오늘의 과정을 통해 추천시스템의 기본 원리를 파악해 보고 나만의 추천시스템을 만들 수 있는 멋진 아이디어를 떠올려 봅시다.  
  
실습 목표
0. 추천시스템의 개념과 목적을 이해한다.
1. Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다.
2. 음악 감상 기록을 활용하여 비슷한 아티스트를 찾고 아티스트를 추천해 본다.
3. 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다
4. 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.
5. 새로운 데이터셋으로 직접 추천 모델을 만들어 본다.  

목차
1. 들어가며
2. 데이터 탐색하기와 전처리
3. 사용자의 명시적/암묵적 평가
4. Matrix Factorization(MF)
5. CSR(Compressed Sparse Row) Matrix
6. MF 모델 학습하기
7. 비슷한 아티스트 찾기 + 유저에게 추천하기
8. 프로젝트 - MovieLens 영화 추천 실습  

  
# 8-2. 추천시스템이란 게 뭔가요?  
추천시스템은 이처럼 a)아이템은 매우 많고 b)유저의 취향은 다양할 때 유저가 소비할만한 아이템을 예측하는 모델입니다.  
추천 기반의 미디어 회사들은 꾸준히 아이템이 되는 아티스트 정보를 자동으로 수집하고, 사용자들이 아티스트를 선택한 정보를 업데이트하여 축적하고 있습니다. 이렇게 쌓인 방대한 데이터가 정확한 협업 필터링을 위한 기반이 됩니다.  

# 8-3. 데이터 탐색하기와 전처리
**데이터 준비**  
1.6Gb가 넘어가는 이렇게 큰 데이터를 처리하는데도 우리의 Pandas는 끄덕없다  
우리가 다룰 데이터는 그동안 흔히 보던 csv 파일이 아니고 tsv 파일입니다. 어떻게 생긴 파일일까요? 다음과 같이 열어 봅시다. 혹시나 해서 당부드립니다만, 굳이 vi를 사용해서 1.6Gb 짜리 데이터 전체를 통째로 메모리 상에 열어보지는 않도록 합시다.  
$ more ~/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv  
tsv는 Tab-Separated Values의 약자로서, Comma-Separated Values의 약자인 csv 파일과 거의 동일하나 구분자만 comma가 아니라 tab('\t') 문자를 활용한 것  
이런 텍스트 파일은 Pandas의 read_csv 메소드로도 충분히 파싱해서 열어볼 수 있습니다.  
  
위 데이터에는 컬럼명이 포함되어 있지 않으므로, 아래와 같이 col_names에 적절히 컬럼명을 지정해 주도록 하겠습니다.  

In [1]:
import pandas as pd
import os

fname = os.getenv('HOME') + '/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 지정한 컬럼명
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


In [2]:
# artist_MBID 항목은 우리의 관심사가 아닙니다. 사용하는 컬럼만 남기고 제거하도록 하겠습니다
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [3]:
# 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data['artist'] = data['artist'].str.lower() 
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [4]:
# 첫 번째 유저가 어떤 아티스트의 노래를 듣는지 확인해 봅시다.
condition = (data['user_id']== data.loc[0, 'user_id'])
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


추천시스템에서는 적용하는 분야에 대한 지식이 특히 더 필요한데, 모르면 검증을 못 하기 때문입니다.  
  
**데이터 탐색**  
추천 모델을 만들기 전에 데이터의 기본적인 정보를 보고 갑시다. 아래 항목들을 확인해 보고 싶습니다.  
+ 유저수, 아티스트수, 인기 많은 아티스트
+ 유저들이 몇 명의 아티스트를 듣고 있는지에 대한 통계
+ 유저 play 횟수 중앙값에 대한 통계  
(참고) pandas.DataFrame.nunique()은 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용합니다.

In [5]:
# 유저 수
data['user_id'].nunique()

358868

In [6]:
# 아티스트 수
data['artist'].nunique()

291346

In [7]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [8]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [9]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

**모델 검증을 위한 사용자 초기 정보 세팅**  
유튜브 뮤직 등 추천시스템들은 이를 위해서 처음 가입하는 사용자의 취향과 유사한 아티스트 정보를 5개 이상 입력받는 과정을 거치게 하는 경우가 많습니다.  
우리도 동일한 과정이 필요합니다.   
우리가 만든 추천시스템의 추후 검증 과정을 위해, '나는 A를 좋아한다.'라는 정보를 위 데이터셋에 5개 이상 추가해 주어야 합니다. 저는 아래와 같이 제가 좋아하는 유명한 아티스트들을 기존의 데이터에 추가해서 넣겠습니다.  

In [10]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,black eyed peas,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30


**모델에 활용하기 위한 전처리 (실습)**  
데이터의 관리를 쉽게 하기 위해 번호를 붙여주고 싶습니다. 우리가 다루는 데이터에서는 user와 artist 각각에 번호를 붙이고 싶습니다.  
보통 이런 작업을 indexing이라고 합니다. 추천시스템, 자연어처리에서 자주 하는 작업들이라 직접 해보시면 좋겠습니다.  
(참고) pandas.DataFrame.unique()은 특정 컬럼에 포함된 유니크한 데이터만 모아 줍니다. indexing 작업을 위해 매우 유용합니다.  


In [11]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [12]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(artist_to_idx['black eyed peas'])

358868
376


In [13]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data # data의 user_id와 artist 컬럼 내 값들이 모두 정수 인덱스 값으로 잘 변경 => 훈련을 위한 전처리 완료!

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,376,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


# 8-4. 사용자의 명시적/암묵적 평가
사용자의 아이템 선호도를 말해 주는 유저 행동 데이터셋이 필요합니다. 만약 우리 데이터가 좋아요나 별점처럼 선호도를 명시적(explicit)으로 나타내는 것이라면 참 좋겠지만, 지금 우리가 가지고 있는 데이터는 사용자가 아티스트의 곡을 몇 번 플레이했나 하는 것뿐입니다.(묵시적일 수 밖에 없다.)  
  
우리의 데이터셋에서 1회만 플레이한 데이터의 비율

In [14]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

147740,17535660
Ratio of only_one over all data is 0.84%


우리가 앞으로 만들어갈 모델에서는 암묵적 데이터의 해석을 위해 다음과 같은 규칙을 적용할 것입니다.  
  
한 번이라도 들었으면 선호한다고 판단한다.   
많이 재생한 아티스트에 대해 가중치를 주어서 더 확실히 좋아한다고 판단한다.  
  
# 8-5. Matrix Factorization(MF)  
데이터 준비가 끝났으니 모델을 준비하자
![출처: CuMF_SGD: Fast and Scalable Matrix Factorization(https://arxiv.org/pdf/1610.05838.pdf)](https://aiffelstaticprd.blob.core.windows.net/media/images/E-3v2-2_ekCv9hW.max-800x600.png)  
우리는 m명의 사용자들(행)이 n명의 아티스트(열)에 대해 평가한 데이터를 포함한 (m,n) 사이즈의 평가행렬(Rating Matrix)을 만들 것입니다.  
행렬 중 일부는 데이터가 채워져 있지만, 나머지 부분은 데이터가 비어 있습니다. 우리가 앞에서 살펴보았던 추천시스템의 협업 필터링(Collaborative Filtering)이란 결국은 이런 평가행렬을 전제로 하는 것입니다.  
이 평가행렬의 비어 있는 부분을 포함한 완벽한 정보를 얻기 위해 학습시켜야 한다.  
  
우리는 추천 시스템의 다양한 모델 중 Matrix Factorization(MF, 행렬분해) 모델을 사용하겠습니다. MF 모델은 2006년 Netflix에서 백만달러의 상금을 걸고 개최한 자사 추천시스템의 성능을 10% 이상 향상시키는 챌린지를 계기로 알려지게 되었습니다.  
  
(m,n) 사이즈의 행렬 R을 (m,k) 사이즈의 행렬 P와 (k,n) 사이즈의 행렬 Q로 분해한다면 R이란 그저 P와 Q의 행렬곱으로 표현 가능할 수 있다는 간단한 아이디어  
k는 m이나 n보다 훨씬 작은 값이기 때문에 계산량 측면으로도 훨씬 유리  
아이디어의 단순함에도 불구하고 MF 모델은 성능이 준수하고 Scalability가 좋아서 많이 사용되는 모델입니다.  
  
MF 모델은 큰 평가행렬 R을 두 개의 Feature Matrix P와 Q로 분해한다고 하였습니다. 무엇의 Feature일까요?  
![](https://aiffelstaticprd.blob.core.windows.net/media/images/E-3v2-3.max-800x600.png)

(m, k) 사이즈의 Feature Matrix P는 k 차원의 벡터를 사용자 수만큼 모아놓은 행렬입니다. 그렇다면 아래 그림에서 첫 번째 벡터 P0=(1, 0.1)은 바로 빨간 모자를 쓴 첫 번째 사용자의 특성(Feature) 벡터가 됩니다.  
같은 방식으로 Q 행렬의 첫번째 벡터 Q0=(0.9, -0.2)는 해리포터 영화의 특성 벡터가 되겠지요.   
MF 모델은 이 두 벡터를 내적해서 얻어지는 0.88이 바로 R0,0 으로 정의되는 사용자의 영화 선호도로 보는 모델입니다.  
  
모델의 목표는 모든 유저와 아이템에 대해 k-dimension의 벡터를 잘 만드는 것입니다.  
벡터를 잘 만드는 기준은 유저i의 벡터(Ui)와 아이템j의 벡터(Ij)를 내적했을 때 유저i가 아이템j에 대해 평가한 수치(Mij)와 비슷한지 입니다.  
  
MF에도 다양한 변형이 있습니다. 이번에 사용할 모델은 Collaborative Filtering for Implicit Feedback Datasets 논문에서 제안한 모델을 사용할 것입니다.  
이 모델의 경우 이전 스텝에서 제가 'black eyed peas'를 play했다는 데이터를 추가했기 때문에 저의 벡터와 black eyed peas의 벡터를 곱했을 때 1에 가까워야 모델이 잘 학습하는 것을 목표로 합니다.  
  
오해할 수 있는데, 제가 black eyed peas를 5번 들었어도 모델이 근사하고자 하는 것이 아티스트를 선호하는지 안 하는지를 맞추는 것이기 때문에 두 벡터를 곱했을 때 5에 가까워지는 대신 1에 가까워져야 합니다. 이를 통해서 제가 들어보지 못한 queen에 대해서도 저의 벡터와 queen의 벡터를 곱하여 수치를 예상할 수 있습니다.  

만약 목표가 유저의 재생 횟수를 맞춰야 하는 것(혹은 별점을 맞추는 것)이라면 그에 맞는 다른 모델을 사용해야 합니다.  
궁금하다면 https://towardsdatascience.com/recommendation-system-matrix-factorization-d61978660b4b  
  
# 8-6. CSR(Compressed Sparse Row) Matrix
유저 X 아이템 평가행렬을 다시 생각해 봅시다. 유저는 36만 명이고 아티스트는 29만 명입니다. 이를 행렬로 표현하고 행렬의 각 원소에 정수 한 개 (1byte)가 들어간다면 36만 * 29만 * 1byte ≈ 97GB가 필요합니다.  
참고로 여러분들이 사용하고 계실 컴퓨터의 메모리는 많아야 16GB일 테니 97GB나 되는 거대한 행렬을 메모리에 올려놓고 작업한다는 것은 불가능할 것입니다. 이런 경우의 좋은 대안이 되는 것이 CSR(Compressed Sparse Row) Matrix입니다. 다음 스텝에서 수행할 모델 학습의 input으로 사용할 데이터 타입을 CSR Matrix로 할 것입니다.  

이제 우리의 data를 CSR Matrix에 맞게 바꿔보겠습니다.  
```  
csr_matrix ((data, (row_ind, col_ind)), \[shape = (M, N)\])
어디서 data, row_ind그리고 col_ind관계를 만족시킵니다 .a\[row_ind[k], col_ind[k]\] = data\[k\]

***
csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])  
where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k]., M,N은 matrix의 shape
```



In [15]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291347 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Row format>

# 8-7. MF 모델 학습하기

Matrix Factorization 모델을 implicit 패키지를 사용하여 학습해 봅시다.  
이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용하겠습니다. Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있습니다.  


In [16]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 __init__ 파라미터를 살펴보겠습니다.  
1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
3. use_gpu : GPU를 사용할 것인지 
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [17]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [18]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291347x358869 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Column format>

In [19]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

모델이 1)저의 벡터와 black eyed peas의 벡터를 어떻게 만들고 있는지와 2)두 벡터를 곱하면 어떤 값이 나오는지 살펴보겠습니다.

In [39]:
artist_to_idx

{'betty blowtorch': 0,
 'die ärzte': 1,
 'melissa etheridge': 2,
 'elvenking': 3,
 'juliette & the licks': 4,
 'red hot chili peppers': 5,
 'magica': 6,
 'the black dahlia murder': 7,
 'the murmurs': 8,
 'lunachicks': 9,
 'walls of jericho': 10,
 'letzte instanz': 11,
 'goldfrapp': 12,
 'horrorpops': 13,
 'the butchies': 14,
 'jack off jill': 15,
 'babes in toyland': 16,
 'dropkick murphys': 17,
 'all:my:faults': 18,
 'le tigre': 19,
 'schandmaul': 20,
 'edguy': 21,
 'maximum the hormone': 22,
 'all ends': 23,
 'jack johnson': 24,
 'eluveitie': 25,
 'rasputina': 26,
 'london after midnight': 27,
 'the killers': 28,
 'mutyumu': 29,
 'judas priest': 30,
 'rob zombie': 31,
 'the bosshoss': 32,
 'blue öyster cult': 33,
 'sandra nasic': 34,
 'john mayer': 35,
 'sleater-kinney': 36,
 'the who': 37,
 'disciple': 38,
 'tanzwut': 39,
 'guano apes': 40,
 'the rolling stones': 41,
 'little big town': 42,
 'team dresch': 43,
 'betty': 44,
 'l7': 45,
 'bif naked': 46,
 'girlschool': 47,
 'the wallf

In [20]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

print('슝=3')

슝=3


In [21]:
zimin_vector

array([ 4.8020339e-01, -1.5829219e-01,  6.0759282e-01, -1.0294137e+00,
       -8.0009013e-01,  6.2251526e-01,  1.3827977e-01,  1.2845320e+00,
       -7.3684639e-01,  2.7951610e-01,  1.5458294e+00, -1.1100987e+00,
       -1.6680212e-01,  6.4477110e-01, -4.1607389e-01, -2.7390670e-02,
       -6.7079253e-03, -5.2548867e-01, -1.1525543e+00, -2.7240673e-01,
       -5.1357299e-01,  2.5291163e-01,  1.7721143e-01,  1.0100980e+00,
       -7.7293742e-01,  7.6230091e-01, -6.0310483e-01, -4.3191126e-01,
       -6.1851925e-01, -1.0234407e+00, -7.5024545e-02,  4.0642998e-01,
        5.7256669e-01,  6.6694474e-01,  3.5039523e-01,  4.1714272e-01,
        1.7622391e+00,  3.2153454e-01,  1.7982006e-01,  3.5690079e-03,
        4.6346450e-01, -7.0396960e-01, -2.4436752e-01,  2.8694540e-01,
       -7.1001142e-01, -1.7799450e-02,  9.3046480e-01,  7.0193821e-01,
       -8.1740296e-01,  6.6880107e-01, -4.1519699e-01, -2.9262209e-01,
       -5.2287900e-01, -7.4204546e-01, -1.1823319e+00, -3.5898438e-01,
      

In [22]:
black_eyed_peas_vector

array([ 0.01518469,  0.0081678 , -0.00128484, -0.0164303 , -0.00309874,
        0.01238807, -0.0017997 ,  0.02550986,  0.01124998,  0.01242891,
        0.04101653, -0.00658203,  0.01404013,  0.01331088,  0.00732692,
        0.01057435, -0.00109461, -0.0009129 ,  0.00072273, -0.00458813,
        0.00779519,  0.00714884,  0.00257136,  0.02683359, -0.01000379,
        0.03261067, -0.00663731,  0.00262288, -0.00257129, -0.00670778,
        0.01290764,  0.01583477,  0.01687097,  0.04118395,  0.01322683,
        0.03182806,  0.03649414,  0.02142146,  0.01697619,  0.00768097,
        0.00821959,  0.01114766,  0.02552001,  0.00361793,  0.02101965,
        0.0184805 ,  0.00498513,  0.0123626 ,  0.00061317,  0.01767504,
        0.00593456, -0.00396662,  0.00453833, -0.00898183, -0.01326058,
       -0.00788477,  0.00871379,  0.00255659,  0.01078339,  0.01147169,
        0.01809497,  0.01215265,  0.01738272,  0.01122583, -0.01267943,
        0.01010958,  0.00672496,  0.00194361,  0.01394734, -0.00

In [23]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(zimin_vector, black_eyed_peas_vector)

0.47934452

factors를 늘리거나 iterations를 늘려서 1에 가깝게 나왔다면 이 모델이 잘 학습되었다고 볼 수 있을까요?  
아직 검증되지 않았다입니다. 학습 데이터에 대해서 fitting은 되었지만 아직 보지 못한 데이터에 대한, 예를 들어 제가 queen을 선호할지에 대한 예측을 잘할지 못할지는 검증하지 않았기 때문입니다.  

In [24]:
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.27560404

# 8-8. 비슷한 아티스트 찾기 + 유저에게 추천하기
**비슷한 아티스트 찾기**  
AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 아티스트를 찾습니다. 처음으로는 제가 좋아하는 coldplay로 찾아보겠습니다.  



In [25]:
# (아티스트의 id, 유사도) Tuple 로 반환
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

[(62, 1.0),
 (277, 0.98425084),
 (28, 0.9819824),
 (473, 0.9731282),
 (5, 0.9711886),
 (217, 0.9704095),
 (490, 0.9679893),
 (418, 0.9617712),
 (247, 0.9588625),
 (694, 0.951834),
 (55, 0.9497125),
 (1018, 0.9460387),
 (910, 0.94070977),
 (782, 0.938629),
 (1500, 0.9336708)]

In [26]:
# 아티스트의 id를 다시 아티스트의 이름으로 매핑
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'the killers',
 'placebo',
 'red hot chili peppers',
 'radiohead',
 'oasis',
 'u2',
 'the beatles',
 'foo fighters',
 'arctic monkeys',
 'the smashing pumpkins',
 'nirvana',
 'the white stripes',
 'gorillaz']

In [38]:
idx_to_artist

{0: 'betty blowtorch',
 1: 'die ärzte',
 2: 'melissa etheridge',
 3: 'elvenking',
 4: 'juliette & the licks',
 5: 'red hot chili peppers',
 6: 'magica',
 7: 'the black dahlia murder',
 8: 'the murmurs',
 9: 'lunachicks',
 10: 'walls of jericho',
 11: 'letzte instanz',
 12: 'goldfrapp',
 13: 'horrorpops',
 14: 'the butchies',
 15: 'jack off jill',
 16: 'babes in toyland',
 17: 'dropkick murphys',
 18: 'all:my:faults',
 19: 'le tigre',
 20: 'schandmaul',
 21: 'edguy',
 22: 'maximum the hormone',
 23: 'all ends',
 24: 'jack johnson',
 25: 'eluveitie',
 26: 'rasputina',
 27: 'london after midnight',
 28: 'the killers',
 29: 'mutyumu',
 30: 'judas priest',
 31: 'rob zombie',
 32: 'the bosshoss',
 33: 'blue öyster cult',
 34: 'sandra nasic',
 35: 'john mayer',
 36: 'sleater-kinney',
 37: 'the who',
 38: 'disciple',
 39: 'tanzwut',
 40: 'guano apes',
 41: 'the rolling stones',
 42: 'little big town',
 43: 'team dresch',
 44: 'betty',
 45: 'l7',
 46: 'bif naked',
 47: 'girlschool',
 48: 'the w

In [27]:
# 몇 번 더 반복해서 확인하기 위해 위의 코드를 함수로 만들고 확인
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

print("슝=3")

슝=3


In [28]:
get_similar_artist('2pac')

['2pac',
 'notorious b.i.g.',
 'dr. dre',
 'nas',
 'snoop dogg',
 'jay-z',
 'the game',
 '50 cent',
 'busta rhymes',
 'mobb deep']

마니아는 데이터상에서 표현될 때 어떤 특징을 가질까요?  
마니아들은 특정 장르의 아티스트들에게로 선호도가 집중되고, 다른 장르의 아티스트들과는 선호도가 낮게 나타날 것입니다. 이런 마니아들의 존재로 인해 같은 장르의 아티스트들의 벡터들도 더 가까워져서 get_similar_artist시 장르별 특성이 두드러지게 될 것입니다.

In [29]:
get_similar_artist('lady gaga')

['lady gaga',
 'britney spears',
 'rihanna',
 'katy perry',
 'beyoncé',
 'the pussycat dolls',
 'nelly furtado',
 'christina aguilera',
 'leona lewis',
 'pink']

**유저에게 아티스트 추천하기**  
AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 제가 좋아할 만한 아티스트를 추천받습니다.  filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.  

In [30]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(350, 0.4531334),
 (627, 0.4478812),
 (550, 0.4412536),
 (354, 0.433991),
 (382, 0.43300325),
 (1800, 0.40991664),
 (621, 0.40972185),
 (355, 0.40875372),
 (391, 0.40844095),
 (369, 0.40693372),
 (901, 0.40007502),
 (409, 0.39957815),
 (274, 0.39601672),
 (2249, 0.38822258),
 (944, 0.36730158),
 (2902, 0.3669501),
 (5555, 0.36525893),
 (724, 0.36130238),
 (618, 0.35853612),
 (3005, 0.35748467)]

In [31]:
[idx_to_artist[i[0]] for i in artist_recommended]

['rihanna',
 'maroon 5',
 'britney spears',
 'nelly furtado',
 'mika',
 'lady gaga',
 'alicia keys',
 'madonna',
 'christina aguilera',
 'justin timberlake',
 'pink',
 'amy winehouse',
 'michael jackson',
 'katy perry',
 'avril lavigne',
 'james blunt',
 'timbaland',
 'lily allen',
 'the pussycat dolls',
 'duffy']

rihanna를 추천해주고 있네요. 모델은 왜 rihanna를 추천해줬을까요?  
AlternatingLeastSquares 클래스에 구현된 explain 메소드를 사용하면 제가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있습니다.  

In [32]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

# 이 method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)를 반환합니다
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('beyoncé', 0.23209677880178733),
 ('black eyed peas', 0.12979297226461198),
 ('jason mraz', 0.06035735329884432),
 ('coldplay', 0.03261624413067068),
 ('maroon5', -2.1892978591528262e-05)]

눈치채신 분도 있겠지만 결과가 조금 이상합니다. filter_already_liked_items=True로 했는데 앞에 maroon 5는 제가 이미 평가한 아이템이 나오고 있습니다. 이것은 implicit 버전 0.4.2에서 생긴 버그입니다. 코드는 사람이 만들기 때문에 이렇게 실수할 가능성이 있습니다.  
**마무리**  
우리가 만든 모델의 몇 가지 아쉬운 점

유저, 아티스트에 대한 Meta 정보를 반영하기 쉽지 않습니다. 연령대별로 음악 취향이 굉장히 다르지 않을까요?
유저가 언제 play 했는지에 반영하기 쉽지 않습니다. 10년 전에 즐겨듣던 아티스트와 지금 즐겨듣는 아티스트를 비교해보세요.  
  
# 8-9. 프로젝트 - Movielens 영화 추천 실습
이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.  
이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.  

+ 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
+ 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
+ 별점을 시청횟수로 해석해서 생각하겠습니다.
+ 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
MovieLens 1M Dataset 데이터셋의 경우는 크기가 작아서 아래와 같이 직접 다운 받고 압축해제 하겠습니다.

## **1) 데이터 준비와 전처리**  
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.



In [33]:
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [34]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [35]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [36]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [37]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head() # 여기까지가 전처리

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2) 분석해 봅시다.  
ratings에 있는 유니크한 영화 개수  
rating에 있는 유니크한 사용자 수  
가장 인기 있는 영화 30개(인기순)  
## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.  
## 4) CSR matrix를 직접 만들어 봅시다.  
## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.   
## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.  
## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.  
## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.
  
평가문항	상세기준
1. CSR matrix가 정상적으로 만들어졌다.  
사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.  
  
  
2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.  
사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.  
   
   
3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.  
MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.  
